## Checkpoints Analysis on Keras using Microscopy Dataset

In [1]:
# Import related libraries
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import  img_to_array,  load_img

In [2]:
SIZE = 150
###2 conv and pool layers. with some normalization and drops in between.
INPUT_SHAPE = (SIZE, SIZE, 3) 

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [4]:
batch_size = 10

In [5]:
# Prepare Data
# Training augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Create Generator 
# This will read pictures found in subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/cell_images/train',  # directory
        target_size=(150, 150),  
        batch_size=batch_size,
        class_mode='binary')  

Found 5600 images belonging to 2 classes.


In [7]:
# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        'data/cell_images/test',  # directory
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 100 images belonging to 2 classes.


In [8]:
#Add Callbacks: ModelCheckpoints
from keras.callbacks import ModelCheckpoint

In [9]:
#ModelCheckpoint callback saves a model at some interval.
filepath="saved_models/jupnote_model.{epoch:02d}-{val_accuracy:.2f}.h5" #File name includes epoch and validation accuracy.
checkpoint = ModelCheckpoint(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode='max')

In [10]:
# Add our checkpoint for callback list, may choose to add early stopping soon..
callbacks_list = [checkpoint]

In [11]:
#We can now use these generators to train our model.
history = model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=6,
        validation_data=validation_generator,
        callbacks=callbacks_list)
model.save('malaria_augmented_model.h5')  # always save your weights after training or during training

Epoch 1/6
200/200 [==============================] - ETA: 0s - loss: 0.3696 - accuracy: 0.8490
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to saved_models\jupnote_model.01-0.60.h5
200/200 [==============================] - 35s 169ms/step - loss: 0.3696 - accuracy: 0.8490 - val_loss: 5.2225 - val_accuracy: 0.6000
Epoch 2/6
200/200 [==============================] - ETA: 0s - loss: 0.2653 - accuracy: 0.9865
Epoch 2: val_accuracy improved from 0.60000 to 0.64000, saving model to saved_models\jupnote_model.02-0.64.h5
200/200 [==============================] - 33s 164ms/step - loss: 0.2653 - accuracy: 0.9865 - val_loss: 3.6381 - val_accuracy: 0.6400
Epoch 3/6
200/200 [==============================] - ETA: 0s - loss: 0.2114 - accuracy: 0.9920
Epoch 3: val_accuracy did not improve from 0.64000
200/200 [==============================] - 33s 165ms/step - loss: 0.2114 - accuracy: 0.9920 - val_loss: 3.8347 - val_accuracy: 0.6000
Epoch 4/6
200/200 [==========================

In [12]:
train_loss, train_acc = model.evaluate(train_generator, steps=15)
validation_loss, test_acc = model.evaluate(validation_generator, steps=10)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

10/10 [==============================] - 0s 32ms/step - loss: 9.3358 - accuracy: 0.6000
Train: 1.000, Test: 0.600


Let us try reloading the exported model and check for accuracy againts the validation set

In [25]:
from keras.models import load_model
loaded_model = load_model("saved_models/jupnote_model.05-0.72.h5")

In [26]:
loaded_model.evaluate(validation_generator, steps=10)

10/10 [==============================] - 0s 30ms/step - loss: 3.9511 - accuracy: 0.7200


[3.9510796070098877, 0.7200000286102295]

In [27]:
loaded_model.evaluate(train_generator, steps=15)

15/15 [==============================] - 1s 59ms/step - loss: 3.9596e-09 - accuracy: 1.0000


[3.959585104240659e-09, 1.0]

In [28]:
loaded_model.save('saved_models/loaded_model_malaria_augmented_model.h5') 
loaded_model_2 = load_model("saved_models/loaded_model_malaria_augmented_model.h5")

In [29]:
all_generator = validation_datagen.flow_from_directory(
        'data/cell_images/all',  # directory
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 5700 images belonging to 2 classes.


In [30]:
loaded_model_2.evaluate(all_generator, steps=15)

15/15 [==============================] - 1s 31ms/step - loss: 0.0788 - accuracy: 0.9933


[0.0787973701953888, 0.9933333396911621]